In [16]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding
os.chdir('C:/Users/kingsubham27091995/Desktop/AppliedAiCouse/CASE STUDIES/MusicGeneration/Assignment/Music-Generation-Using-Deep-Learning-master')

In [17]:
data_directory = "./Data2/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = './Data2/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [18]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i] + 1. 
        yield X, Y

In [25]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True, name = "lstm_first"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    model.load_weights("./Data2/Model_Weights/Weights_90.h5", by_name = True)
    
    return model

In [26]:
def training_model(data, epochs = 90):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("./Data2/log.csv", index = False)

In [27]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 87
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embd_1 (Embedding)           (16, 64, 512)             44544     
_________________________________________________________________
lstm_first (LSTM)            (16, 64, 256)             787456    
_________________________________________________________________
drp_1 (Dropout)              (16, 64, 256)             0         
_________________________________________________________________
lstm_9 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_9 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_10 (LSTM)               (16, 64, 256)             525312    
_________________________________________________________________
dropout_10 (Dro

Batch: 104, Loss: 1.5608556270599365, Accuracy: 0.5546875
Batch: 105, Loss: 1.643567442893982, Accuracy: 0.533203125
Batch: 106, Loss: 1.660261631011963, Accuracy: 0.5126953125
Batch: 107, Loss: 1.773576259613037, Accuracy: 0.5087890625
Batch: 108, Loss: 1.7740228176116943, Accuracy: 0.490234375
Batch: 109, Loss: 1.719454288482666, Accuracy: 0.51171875
Batch: 110, Loss: 1.4477866888046265, Accuracy: 0.578125
Batch: 111, Loss: 1.5840625762939453, Accuracy: 0.5146484375
Batch: 112, Loss: 1.6169992685317993, Accuracy: 0.5458984375
Batch: 113, Loss: 1.6859430074691772, Accuracy: 0.552734375
Batch: 114, Loss: 1.6559414863586426, Accuracy: 0.5283203125
Batch: 115, Loss: 1.7547707557678223, Accuracy: 0.5224609375
Batch: 116, Loss: 1.7020313739776611, Accuracy: 0.52734375
Batch: 117, Loss: 1.702903151512146, Accuracy: 0.53125
Batch: 118, Loss: 1.530562400817871, Accuracy: 0.576171875
Batch: 119, Loss: 1.5619195699691772, Accuracy: 0.5634765625
Batch: 120, Loss: 1.5767087936401367, Accuracy: 0.

Batch: 92, Loss: 1.2635743618011475, Accuracy: 0.623046875
Batch: 93, Loss: 1.1668829917907715, Accuracy: 0.6484375
Batch: 94, Loss: 1.1440253257751465, Accuracy: 0.650390625
Batch: 95, Loss: 1.1791906356811523, Accuracy: 0.62890625
Batch: 96, Loss: 1.1939702033996582, Accuracy: 0.6376953125
Batch: 97, Loss: 1.0997185707092285, Accuracy: 0.669921875
Batch: 98, Loss: 1.1126652956008911, Accuracy: 0.6650390625
Batch: 99, Loss: 1.1076678037643433, Accuracy: 0.6591796875
Batch: 100, Loss: 1.1233456134796143, Accuracy: 0.6552734375
Batch: 101, Loss: 1.2073593139648438, Accuracy: 0.6328125
Batch: 102, Loss: 1.1007065773010254, Accuracy: 0.6708984375
Batch: 103, Loss: 1.2402665615081787, Accuracy: 0.6513671875
Batch: 104, Loss: 1.1061865091323853, Accuracy: 0.646484375
Batch: 105, Loss: 1.1939175128936768, Accuracy: 0.634765625
Batch: 106, Loss: 1.1763380765914917, Accuracy: 0.6455078125
Batch: 107, Loss: 1.3100272417068481, Accuracy: 0.62109375
Batch: 108, Loss: 1.302626371383667, Accuracy: 

Batch: 80, Loss: 0.9206836223602295, Accuracy: 0.7060546875
Batch: 81, Loss: 1.0645498037338257, Accuracy: 0.6513671875
Batch: 82, Loss: 1.049135446548462, Accuracy: 0.669921875
Batch: 83, Loss: 0.8926960825920105, Accuracy: 0.7451171875
Batch: 84, Loss: 1.0084192752838135, Accuracy: 0.7021484375
Batch: 85, Loss: 0.9766154289245605, Accuracy: 0.7119140625
Batch: 86, Loss: 1.2148382663726807, Accuracy: 0.640625
Batch: 87, Loss: 0.9541926980018616, Accuracy: 0.720703125
Batch: 88, Loss: 1.1125962734222412, Accuracy: 0.6865234375
Batch: 89, Loss: 1.079204797744751, Accuracy: 0.693359375
Batch: 90, Loss: 0.9872555732727051, Accuracy: 0.7021484375
Batch: 91, Loss: 1.0333572626113892, Accuracy: 0.6962890625
Batch: 92, Loss: 1.0608742237091064, Accuracy: 0.681640625
Batch: 93, Loss: 0.9994813799858093, Accuracy: 0.701171875
Batch: 94, Loss: 1.0021504163742065, Accuracy: 0.6787109375
Batch: 95, Loss: 1.0370919704437256, Accuracy: 0.677734375
Batch: 96, Loss: 1.0349397659301758, Accuracy: 0.678

Batch: 68, Loss: 1.0938313007354736, Accuracy: 0.6640625
Batch: 69, Loss: 0.9884229898452759, Accuracy: 0.6904296875
Batch: 70, Loss: 0.9735668897628784, Accuracy: 0.7216796875
Batch: 71, Loss: 1.0047624111175537, Accuracy: 0.693359375
Batch: 72, Loss: 0.8777171969413757, Accuracy: 0.7265625
Batch: 73, Loss: 0.9067394733428955, Accuracy: 0.728515625
Batch: 74, Loss: 0.8640621900558472, Accuracy: 0.732421875
Batch: 75, Loss: 0.81492680311203, Accuracy: 0.7529296875
Batch: 76, Loss: 0.885761022567749, Accuracy: 0.720703125
Batch: 77, Loss: 0.9085817337036133, Accuracy: 0.7109375
Batch: 78, Loss: 0.9478323459625244, Accuracy: 0.703125
Batch: 79, Loss: 0.894633948802948, Accuracy: 0.736328125
Batch: 80, Loss: 0.835137128829956, Accuracy: 0.732421875
Batch: 81, Loss: 0.9826111197471619, Accuracy: 0.6943359375
Batch: 82, Loss: 0.9515863656997681, Accuracy: 0.7080078125
Batch: 83, Loss: 0.8178856372833252, Accuracy: 0.765625
Batch: 84, Loss: 0.9510161876678467, Accuracy: 0.71484375
Batch: 85,

...

Batch: 49, Loss: 0.20103681087493896, Accuracy: 0.9326171875
Batch: 50, Loss: 0.23300284147262573, Accuracy: 0.927734375
Batch: 51, Loss: 0.20834484696388245, Accuracy: 0.9326171875
Batch: 52, Loss: 0.20086422562599182, Accuracy: 0.9345703125
Batch: 53, Loss: 0.18514463305473328, Accuracy: 0.9384765625
Batch: 54, Loss: 0.2061903178691864, Accuracy: 0.9326171875
Batch: 55, Loss: 0.24781736731529236, Accuracy: 0.9189453125
Batch: 56, Loss: 0.23577767610549927, Accuracy: 0.91796875
Batch: 57, Loss: 0.23191148042678833, Accuracy: 0.9248046875
Batch: 58, Loss: 0.21302935481071472, Accuracy: 0.9296875
Batch: 59, Loss: 0.2324216365814209, Accuracy: 0.9208984375
Batch: 60, Loss: 0.20468127727508545, Accuracy: 0.921875
Batch: 61, Loss: 0.26631543040275574, Accuracy: 0.90625
Batch: 62, Loss: 0.2332151234149933, Accuracy: 0.919921875
Batch: 63, Loss: 0.23499172925949097, Accuracy: 0.921875
Batch: 64, Loss: 0.20439887046813965, Accuracy: 0.9306640625
Batch: 65, Loss: 0.20165473222732544, Accuracy:

Batch: 35, Loss: 0.23425407707691193, Accuracy: 0.91796875
Batch: 36, Loss: 0.24414274096488953, Accuracy: 0.9345703125
Batch: 37, Loss: 0.2268543839454651, Accuracy: 0.923828125
Batch: 38, Loss: 0.2150302231311798, Accuracy: 0.9345703125
Batch: 39, Loss: 0.2465973049402237, Accuracy: 0.921875
Batch: 40, Loss: 0.2449311465024948, Accuracy: 0.9150390625
Batch: 41, Loss: 0.232875257730484, Accuracy: 0.9287109375
Batch: 42, Loss: 0.17142188549041748, Accuracy: 0.9345703125
Batch: 43, Loss: 0.2073156088590622, Accuracy: 0.9248046875
Batch: 44, Loss: 0.1857537031173706, Accuracy: 0.94140625
Batch: 45, Loss: 0.19521306455135345, Accuracy: 0.9384765625
Batch: 46, Loss: 0.2243424504995346, Accuracy: 0.923828125
Batch: 47, Loss: 0.21646589040756226, Accuracy: 0.9248046875
Batch: 48, Loss: 0.20017105340957642, Accuracy: 0.93359375
Batch: 49, Loss: 0.22258169949054718, Accuracy: 0.91796875
Batch: 50, Loss: 0.25090742111206055, Accuracy: 0.912109375
Batch: 51, Loss: 0.17677244544029236, Accuracy: 

Batch: 21, Loss: 0.18986636400222778, Accuracy: 0.931640625
Batch: 22, Loss: 0.24675823748111725, Accuracy: 0.9111328125
Batch: 23, Loss: 0.2586097717285156, Accuracy: 0.919921875
Batch: 24, Loss: 0.25102853775024414, Accuracy: 0.9208984375
Batch: 25, Loss: 0.2305125892162323, Accuracy: 0.923828125
Batch: 26, Loss: 0.20513150095939636, Accuracy: 0.9375
Batch: 27, Loss: 0.21057415008544922, Accuracy: 0.931640625
Batch: 28, Loss: 0.2411976158618927, Accuracy: 0.9150390625
Batch: 29, Loss: 0.22407324612140656, Accuracy: 0.9228515625
Batch: 30, Loss: 0.19588179886341095, Accuracy: 0.9345703125
Batch: 31, Loss: 0.21542063355445862, Accuracy: 0.9267578125
Batch: 32, Loss: 0.23687472939491272, Accuracy: 0.9248046875
Batch: 33, Loss: 0.23478445410728455, Accuracy: 0.927734375
Batch: 34, Loss: 0.23380199074745178, Accuracy: 0.916015625
Batch: 35, Loss: 0.2325403392314911, Accuracy: 0.927734375
Batch: 36, Loss: 0.248733252286911, Accuracy: 0.92578125
Batch: 37, Loss: 0.23842939734458923, Accurac

Batch: 7, Loss: 0.2121715247631073, Accuracy: 0.931640625
Batch: 8, Loss: 0.18473592400550842, Accuracy: 0.935546875
Batch: 9, Loss: 0.20657697319984436, Accuracy: 0.935546875
Batch: 10, Loss: 0.18107661604881287, Accuracy: 0.9345703125
Batch: 11, Loss: 0.2042701542377472, Accuracy: 0.9345703125
Batch: 12, Loss: 0.20317605137825012, Accuracy: 0.9345703125
Batch: 13, Loss: 0.19495412707328796, Accuracy: 0.9345703125
Batch: 14, Loss: 0.22249428927898407, Accuracy: 0.9267578125
Batch: 15, Loss: 0.18864527344703674, Accuracy: 0.9423828125
Batch: 16, Loss: 0.1747516691684723, Accuracy: 0.9462890625
Batch: 17, Loss: 0.16068851947784424, Accuracy: 0.947265625
Batch: 18, Loss: 0.2184918075799942, Accuracy: 0.9345703125
Batch: 19, Loss: 0.2131032645702362, Accuracy: 0.927734375
Batch: 20, Loss: 0.20952989161014557, Accuracy: 0.9345703125
Batch: 21, Loss: 0.1708904206752777, Accuracy: 0.9404296875
Batch: 22, Loss: 0.22550280392169952, Accuracy: 0.9189453125
Batch: 23, Loss: 0.22147195041179657, 

Batch: 144, Loss: 0.2609105110168457, Accuracy: 0.91796875
Batch: 145, Loss: 0.19166702032089233, Accuracy: 0.947265625
Batch: 146, Loss: 0.21562258899211884, Accuracy: 0.931640625
Batch: 147, Loss: 0.19857516884803772, Accuracy: 0.9326171875
Batch: 148, Loss: 0.21773535013198853, Accuracy: 0.9228515625
Batch: 149, Loss: 0.1981217861175537, Accuracy: 0.9345703125
Batch: 150, Loss: 0.2240734100341797, Accuracy: 0.9189453125
Batch: 151, Loss: 0.21611016988754272, Accuracy: 0.921875
Saved Weights at epoch 90 to file Weights_90.h5


In [28]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

,Epoch,Loss,Accuracy
0,1,1.420880,0.601562
1,2,1.100488,0.658203
2,3,0.941171,0.717773
3,4,0.855421,0.736328
4,5,0.805130,0.749023
5,6,0.769024,0.779297
6,7,0.687929,0.785156
7,8,0.698220,0.790039
8,9,0.652558,0.804688
9,10,0.620810,0.814453


## Accuracy= 92% , which is pretty good to generate music
